In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


cache_dir = "/root/autodl-tmp"


from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PrefixEncoder, PrefixTuningConfig
from datasets import Dataset, load_dataset




In [2]:



ds = Dataset.load_from_disk("./alpaca_data_zh")
ds[:1]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'],
 'input': [''],
 'instruction': ['保持健康的三个提示。']}

In [3]:

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path="Langboat/bloom-1b4-zh", cache_dir=os.path.join(cache_dir, "bloom-1b4-zh"))

tokenizer



BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:

# def process_func(example):
#     input_ids, attention_mask, labels =[], [], []
#     instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"].strip() , "\n\nAssistant: "]))
#     response = tokenizer(example["output"] + tokenizer.eos_token)
#     input_ids = instruction["input_ids"] + response["input_ids"]
#     attention_mask = instruction["attention_mask"] + response["attention_mask"]
#     labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]

#     max_length = 256
#     if len(input_ids) > max_length:
#         input_ids = input_ids[:max_length]
#         attention_mask = attention_mask[:max_length]
#         labels = labels[: max_length]
#     return {"input_ids": input_ids, "attention_mask":attention_mask, "labels": labels}


def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }                     


In [5]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)

tokenized_ds[0]

{'input_ids': [26283,
  29,
  210,
  7160,
  28215,
  9905,
  15211,
  672,
  189,
  4340,
  17245,
  29,
  210,
  5040,
  584,
  7160,
  28215,
  9905,
  15211,
  1038,
  189,
  189,
  20,
  17,
  210,
  7160,
  9239,
  3365,
  420,
  9487,
  1965,
  15342,
  9239,
  6053,
  355,
  1227,
  32998,
  554,
  34487,
  1326,
  17632,
  355,
  975,
  5189,
  39177,
  6455,
  355,
  12144,
  17335,
  8481,
  355,
  1437,
  12785,
  6698,
  23294,
  672,
  189,
  21,
  17,
  210,
  22905,
  15356,
  420,
  9487,
  15597,
  22985,
  373,
  19320,
  554,
  18524,
  554,
  1229,
  6663,
  1467,
  642,
  16422,
  18235,
  27624,
  17520,
  8732,
  355,
  8729,
  1246,
  7110,
  554,
  1246,
  16422,
  642,
  15224,
  11367,
  355,
  718,
  7160,
  28215,
  15356,
  12577,
  672,
  189,
  22,
  17,
  210,
  20027,
  18399,
  420,
  20027,
  1079,
  15713,
  6455,
  21230,
  355,
  34810,
  9487,
  1688,
  8637,
  967,
  3958,
  210,
  43738,
  20027,
  420,
  15822,
  20027,
  12785,
  17374,
  12

In [6]:


# 创建模型

model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh", low_cpu_mem_usage=True, cache_dir=os.path.join(cache_dir, "bloom-1b4-zh"))


In [7]:
for name, parameter in model.named_parameters():
    print(name)

transformer.word_embeddings.weight
transformer.word_embeddings_layernorm.weight
transformer.word_embeddings_layernorm.bias
transformer.h.0.input_layernorm.weight
transformer.h.0.input_layernorm.bias
transformer.h.0.self_attention.query_key_value.weight
transformer.h.0.self_attention.query_key_value.bias
transformer.h.0.self_attention.dense.weight
transformer.h.0.self_attention.dense.bias
transformer.h.0.post_attention_layernorm.weight
transformer.h.0.post_attention_layernorm.bias
transformer.h.0.mlp.dense_h_to_4h.weight
transformer.h.0.mlp.dense_h_to_4h.bias
transformer.h.0.mlp.dense_4h_to_h.weight
transformer.h.0.mlp.dense_4h_to_h.bias
transformer.h.1.input_layernorm.weight
transformer.h.1.input_layernorm.bias
transformer.h.1.self_attention.query_key_value.weight
transformer.h.1.self_attention.query_key_value.bias
transformer.h.1.self_attention.dense.weight
transformer.h.1.self_attention.dense.bias
transformer.h.1.post_attention_layernorm.weight
transformer.h.1.post_attention_layernor

In [11]:
"""
创建lora微调的代码和结构
"""

# from peft import LoraConfig, TaskType, get_peft_model
from peft import IA3Config, TaskType, get_peft_model

config = IA3Config(task_type=TaskType.CAUSAL_LM,
                   # 这是针对bloom模型设定好的target_modules,当执行get_peft_model时可以自动获得得到
                   target_modules=["query_key_value", "mlp.dense_4h_to_h"],  # 支持正则表达式
                   feedforward_modules=["mlp.dense_4h_to_h"],
                   modules_to_save=None)

config




IA3Config(peft_type=<PeftType.IA3: 'IA3'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, target_modules={'mlp.dense_4h_to_h', 'query_key_value'}, feedforward_modules={'mlp.dense_4h_to_h'}, fan_in_fan_out=False, modules_to_save=None, init_ia3_weights=True)

In [12]:


# 创建训练参数

from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

args = TrainingArguments(output_dir="/root/autodl-tmp/lora_fine_tuning_model",
                         per_device_train_batch_size=8,
                         gradient_accumulation_steps=4,
                         learning_rate=8e-5, 
                         logging_steps=10, save_total_limit=2, save_steps=50,
                         num_train_epochs=2)

peft_model = get_peft_model(model=model, peft_config=config)

trainer = Trainer(model=peft_model,args = args,
                  data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
                  train_dataset=tokenized_ds)
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 238.00 MiB. GPU 0 has a total capacty of 23.64 GiB of which 160.25 MiB is free. Process 187450 has 12.87 GiB memory in use. Process 192518 has 10.61 GiB memory in use. Of the allocated memory 9.67 GiB is allocated by PyTorch, and 542.38 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:



ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=False)[0], skip_special_tokens=True)




'Human: 考试有哪些技巧？\n\nAssistant: 考试有很多技巧，其中最关键的技巧之一是保持良好的心态。保持良好的心态不仅有助于提高考试成绩，而且有助于保持积极的心态，避免紧张和焦虑。此外，考试技巧还包括合理安排时间，合理分配精力，保持良好的作息习惯，以及合理饮食和锻炼身体。此外，考试技巧还包括保持良好的学习习惯，如按时完成作业，认真听课，及时复习等。'